In [29]:
import os
import spacy
import en_core_web_sm
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import time


In [21]:
spacy_eng = en_core_web_sm.load()
data = os.path.join(os.getcwd(),'data.csv')

In [23]:
# To map each word to a index, convert string to numerical value
class Vocabulary:
    def __init__(self, freq_threshold):
        # freq_threshold check the freq word in text, if 1 , may not important to us
        # <UNK> if the word freq appear is less than threshold, it will map to <UNK>
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)


    # tokenize the caption [I love coffee] -> ["i","love","coffee"]
    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    # build vocab
    def build_vocabulary(self, sentence_list):
        # count each caption how many times a specific word repeated
        # if over the threshold we will include it, else ignore it
        frequencies = {}
        # start with index 4 because we have include the tagging
        idx = 4

        #self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>", 4: "i"}
        #self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3, "i":4}

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1
                # if the word frequence is we want,(we just need to append 1 times only)
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1 # store word to next index

    # convert text into number
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)


        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] # if word is not in library, <UNK>
            for token in tokenized_text
        ]

In [24]:
class Dataset:
    def __init__(self, captions_file, freq_threshold=5):
        self.df = pd.read_csv(captions_file)

        # Get img, caption columns
        self.captions = self.df["Headline"]
        self.results = self.df["Price movement"]

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        # Send all the caption as list
        self.vocab.build_vocabulary(self.captions.tolist())

        self.X = pd.DataFrame()
        for caption in self.captions:
            numericalized_caption = [self.vocab.stoi["<SOS>"]]
            numericalized_caption += self.vocab.numericalize(caption)
            numericalized_caption.append(self.vocab.stoi["<EOS>"])
            self.X.append(np.array(numericalized_caption))

        self.Y = pd.DataFrame()
        for result in self.results:
            tokenizer = {"Down": 0, "Up": 1}
            self.Y.append(tokenizer[result])

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.Y,test_size=0.2)


In [25]:
dataset = Dataset(data)

In [28]:
print(dataset.captions)

0           EVENING 5: Five things you need to know today
1       KLCI retreats on profit taking on final tradin...
2       Cover Story: Delicate balancing act in a tough...
3                                            重量级股遭抛售 马股走低
4                                         2020年马股的最大赢家和输家
                              ...                        
5636               Market rises for the third day of 2010
5637         Glove makers rally on more upside for sector
5638       Tanjong,CIMB, Axiata push FBM KLCI above 1,290
5639       Tanjong,CIMB, Axiata push FBM KLCI above 1,290
5640               Upward trend intact, stock picking key
Name: Headline, Length: 5641, dtype: object


In [ ]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(dataset.X_train, dataset.y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(dataset.X_test)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))

report = classification_report(dataset.y_test, prediction_linear, output_dict=True)

print('positive: ', report['pos'])
print('negative: ', report['neg'])